In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from vnstock import Quote
from vnstock import Finance
from scipy import stats
import time
from typing import Literal
from scipy.interpolate import interp1d
from pyvinecopulib import Bicop, FitControlsBicop
from scipy.optimize import minimize

<h1>I. Thu thap du lieu<h1>

<h2>1. Xac dinh cong ty can de tam<h2>
<h3>Phan nay khong can chay lai nua<h3>

In [ ]:
#Tim ma chung khoan cua cac cong ty
symbol = pd.read_csv('Du lieu.csv')
symbol.columns = symbol.iloc[0]
symbol = symbol.iloc[1::]
interest_industry = list(symbol['Lĩnh vực'].unique())
interest_industry.pop(1)
symbol = symbol[symbol['Lĩnh vực'].isin(interest_industry)]
symbol_name = list(symbol['Công ty'])

In [ ]:
#Phan chia so lan lay du lieu
STT_list = [0,55,110,156]
name_bins = {}
for i in range(len(STT_list)):
    if STT_list[i] == 0:
        continue
    else:
        name_bins[STT_list[i]] = symbol_name[STT_list[i-1]:STT_list[i]]
history_price = pd.DataFrame()

In [ ]:
#Thu thap du lieu ve gia
#Khong can chay o du lieu nay
for i in name_bins[156]:
    pr = Quote(source='VCI',symbol=i)
    price_status = pr.history(start='2010-01-01',end='2025-01-01',interval='1M')
    price_status['time'] = pd.to_datetime(price_status['time'])
    price_status.set_index('time', inplace=True)
    history_price[i] = price_status['close']

<h2>2. Chon loc ra cac co phieu trong so 200 cp co gia tri von hoa lon nhat<h2>
<h3>Phan nay khong can chay lai nua<h3>

In [ ]:
#chia co phieu ra lam 2 thoi ky
history_price = pd.read_csv('price data.csv')
history_price['time'] = pd.to_datetime(history_price['time'])
history_price.set_index('time', inplace=True)
history_price = history_price[history_price.index.year>2015]
history_price.dropna(axis=1,inplace=True)
history_price.to_csv('compare_period1.csv')


<h2>3. Tai du lieu BCTC ve<h2>
<h3>Phan nay khong can chay lai nua<h3>

In [ ]:
#Cac co phieu can lay bao cao tai chinh
data = pd.read_csv('compare_period1.csv')
data['time'] = pd.to_datetime(data['time'])
data.set_index('time',inplace=True)
stock_of_interest = list(data.columns)

In [ ]:
#chia ra thanh cac muc nho hon de lay du lieu
count = 0
stock_trench = {}
number_stock_per_trench = 7
number_of_trench = int(round(len(stock_of_interest)/number_stock_per_trench,0)+1)
separate_support = [min(number_stock_per_trench*i, len(stock_of_interest)) for i in range(number_of_trench)]#cu chon  co phieu 1 danh sach
for i in range(1,len(separate_support)):
    stock_trench[separate_support[i]] = stock_of_interest[separate_support[i-1]:separate_support[i]]
financial_statement_data = pd.DataFrame()#phuc vu cho cell ke tiep
stock_trench

In [ ]:
char_cf = ['Năm', 'Kỳ','Khấu hao TSCĐ',
           'Mua sắm TSCĐ','Tiền thu được từ thanh lý tài sản cố định'  ]
char_is = ['Năm', 'Kỳ','Doanh thu thuần','Thu nhập tài chính','Thu nhập khác','LN trước thuế','Chi phí tiền lãi vay']
char_bs = ['Năm', 'Kỳ','TÀI SẢN NGẮN HẠN (đồng)','Tiền và tương đương tiền (đồng)', 'Giá trị thuần đầu tư ngắn hạn (đồng)','Nợ ngắn hạn (đồng)',
           'Vay và nợ thuê tài chính ngắn hạn (đồng)',
           'Vay và nợ thuê tài chính dài hạn (đồng)',
           'TỔNG CỘNG TÀI SẢN (đồng)','VỐN CHỦ SỞ HỮU (đồng)','LỢI ÍCH CỦA CỔ ĐÔNG THIỂU SỐ']
char_ratio = ['Năm', 'Kỳ','Số CP lưu hành (Triệu CP)']
sleep_time = 40
for t in list(stock_trench.keys())[-2::]:
    for i in stock_trench[t]:
        if i == "DPP":
            continue
        else:
            finance = Finance(source='VCI',symbol=i)
            #bao cao ket qua hoat dong kinh doanh
            income_state = finance.income_statement(lang='vi')
            for j in char_is:
                if j not in income_state.columns:
                    income_state[j]=0
            income_state = income_state[char_is]
            income_state = income_state[income_state['Năm']>2012]
            year = list(income_state['Năm'])
            quarter = list(income_state['Kỳ'])
            time1 = [f'{i}Q{j}' for i,j in zip(year,quarter)]
            time1 = pd.PeriodIndex(time1,freq='Q').to_timestamp(how='end').normalize()
            income_state['time'] = time1
            income_state.set_index('time',inplace=True)
            income_state.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #bang can doi ke toan
            balance_sheet = finance.balance_sheet(lang='vi')
            for j in char_bs:
                if j not in balance_sheet.columns:
                    balance_sheet[j]=0
            balance_sheet = balance_sheet[char_bs]
            balance_sheet = balance_sheet[balance_sheet['Năm']>2012]
            balance_sheet['time'] = time1
            balance_sheet.drop(['Năm','Kỳ'],axis=1, inplace=True)
            balance_sheet.set_index('time', inplace=True)
            #bao cao luu chuyen tien te
            cashflow = finance.cash_flow(lang='vi')
            for j in char_cf:
                if j not in cashflow.columns:
                    cashflow[j] = 0
            cashflow = cashflow[char_cf]
            cashflow = cashflow[cashflow['Năm']>2012]
            cashflow['time'] = time1
            cashflow.set_index('time', inplace=True)
            cashflow.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #so luong co phieu dang luu hanh
            outstanding_share = finance.ratio(lang='vi')
            outstanding_share = outstanding_share.droplevel(level=0,axis=1)
            for j in char_ratio:
                if j not in outstanding_share.columns:
                    outstanding_share[j] = 0
            outstanding_share = outstanding_share[char_ratio]
            outstanding_share = outstanding_share[outstanding_share['Năm']>2012]
            outstanding_share['time']=time1
            outstanding_share.set_index('time',inplace=True)
            outstanding_share.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #sap nhap cac bao cao lai thanh 1 dataframe
            financial_statement_subdata = pd.concat([income_state,balance_sheet,cashflow,outstanding_share], axis=1)
            financial_statement_subdata['ticket'] = i
            #sap nhap cac cong ty lai thanh 1
            if len(financial_statement_data) == 0:
                financial_statement_data = financial_statement_subdata
            else:
                financial_statement_data = pd.concat([financial_statement_data,financial_statement_subdata],axis=0)
    time.sleep(sleep_time)


<h2>4. Xac nhan lai cac cong ty can de tam, dong thoi thu thap du lieu gia<h2>
<h3>Phan nay khong can chay lai nua<h3>

In [ ]:
#lay du lieu
financial_statement_data = pd.read_csv('financial_statements.csv')
financial_statement_data['time'] = pd.to_datetime(financial_statement_data['time'])
financial_statement_data.set_index(['ticket','time'], inplace=True)

In [ ]:
#Phan trench de lay du lieu
stock_of_interest = list(financial_statement_data.index.get_level_values(0).unique())
stock_trench = {}
number_stock_per_trench = 20
number_of_trench = int(round(len(stock_of_interest)/number_stock_per_trench,0)+1)
trench = [min(number_stock_per_trench*i,len(stock_of_interest)) for i in range(number_of_trench)]
l=0
for i in range(1,len(trench)):
    stock_trench[trench[i]] = stock_of_interest[trench[i-1]:trench[i]]
    l += len(stock_of_interest[trench[i-1]:trench[i]])



In [ ]:
input_data = pd.DataFrame()
compare_data = pd.DataFrame()
for t in stock_trench.keys():
    for i in stock_trench[t]:
        his = Quote(source='VCI', symbol=i)
        #du lieu dau vao
        input_subdata = his.history(start='2015-01-01',end='2026-01-01',interval='1M')
        input_subdata['time'] = pd.to_datetime(input_subdata['time'])
        input_subdata.set_index('time',inplace=True)
        input_subdata = input_subdata[['close']]
        input_subdata.columns = [i]
        #Du lieu so sanh
        compare_subdata = his.history(start='2020-01-01',end='2026-01-01',interval='1D')
        compare_subdata['time'] = pd.to_datetime(compare_subdata['time'])
        compare_subdata.set_index('time',inplace=True)
        compare_subdata = compare_subdata[['close']]
        compare_subdata.columns = [i]
        if len(input_data)==0:
            input_data = input_subdata
        else:
            input_data = pd.concat([input_data,input_subdata],axis=1)
        if len(compare_data)==0:
            compare_data = compare_subdata
        else:
            compare_data = pd.concat([compare_data,compare_subdata],axis=1)
    time.sleep(35)
    

In [ ]:
input_data = input_data.dropna(axis=0)
compare_data = compare_data[compare_data.index.year>2020]
input_data.to_csv('input_data.csv')
compare_data.to_csv('compare_data.csv')

In [ ]:
ticket_general_info = pd.read_csv('Du_lieu.csv')
col = ticket_general_info.iloc[0].to_list()
ticket_general_info.columns = col
ticket_general_info = ticket_general_info.drop(index=ticket_general_info.index[0])
ticket_general_info.set_index('Công ty',inplace=True)

In [ ]:
ticket_of_interest = pd.read_csv('compare_data.csv')
ticket_of_interest['time'] = pd.to_datetime(ticket_of_interest['time'])
ticket_of_interest.set_index('time',inplace=True)
info_of_interest_ticket = ticket_general_info.loc[ticket_of_interest.columns]
info_of_interest_ticket.to_csv('info_of_ticket.csv')

<h1>II. Thuat toan phan bo danh muc<h1>

<h2>1. Xay dung du lieu dau vao cho mo hinh<h2>

<h3>1.1. Bo chi tieu dau vao va cac thanh phan phu khac <h3>

In [ ]:
#xu ly du lieu dau vao
raw_data = pd.read_csv('financial_statements.csv')
raw_data['time']= pd.to_datetime(raw_data['time'])
raw_data.set_index(['ticket','time'],inplace=True)
raw_data = raw_data.sort_index(level='time',ascending=True)
raw_data =raw_data

In [ ]:
#Tinh cac chi tieu trung gian 
intermediary_data = raw_data['Doanh thu thuần']+raw_data['Thu nhập tài chính']+raw_data['Thu nhập khác']
intermediary_data = intermediary_data.to_frame(name='dttn')
## chi phí được ghi âm thay vì dương nên cần phải cận thận khi công trừ các chi tiêu này
intermediary_data['ebit'] = raw_data['LN trước thuế'] - raw_data['Chi phí tiền lãi vay']
intermediary_data['interest'] = - raw_data['Chi phí tiền lãi vay']
intermediary_data['dna'] = raw_data['Khấu hao TSCĐ']
intermediary_data['capex'] = -raw_data['Mua sắm TSCĐ'] - raw_data['Tiền thu được từ thanh lý tài sản cố định']
intermediary_data['wc'] = (raw_data['TÀI SẢN NGẮN HẠN (đồng)']-raw_data['Giá trị thuần đầu tư ngắn hạn (đồng)'] - raw_data[
    'Tiền và tương đương tiền (đồng)']) - (raw_data['Nợ ngắn hạn (đồng)']-raw_data['Vay và nợ thuê tài chính ngắn hạn (đồng)'])
intermediary_data['debt'] = raw_data['Vay và nợ thuê tài chính ngắn hạn (đồng)']+raw_data['Vay và nợ thuê tài chính dài hạn (đồng)']
intermediary_data['equity'] = raw_data['VỐN CHỦ SỞ HỮU (đồng)']
intermediary_data['value'] = intermediary_data['debt']+intermediary_data['equity']#day chinh la chi tieu V
intermediary_data['TA'] = raw_data['TỔNG CỘNG TÀI SẢN (đồng)']
#Tao dung cac mau du lieu dau tien, dữ liệu này là kỳ gần nhất đóng vai trò tính toán ra các dữ liệu trong tương lai ví dụ nhu DTTN
latest_time_of_raw_data = raw_data.groupby(level='ticket').last()
latest_time_of_intermediary_data =intermediary_data.groupby(level='ticket').last() 
ind = latest_time_of_raw_data.index
col = ['dttn','wc','minor_rate','out_share','cash','debt']
initial_data_set = np.full(shape=(len(ind),len(col)),fill_value=0)
initial_data_set = pd.DataFrame(initial_data_set,columns=col,index=ind)
initial_data_set['dttn'] = latest_time_of_intermediary_data['dttn']
initial_data_set['wc'] = latest_time_of_intermediary_data['wc']
initial_data_set['minor_rate'] = latest_time_of_raw_data['LỢI ÍCH CỦA CỔ ĐÔNG THIỂU SỐ']/latest_time_of_raw_data['VỐN CHỦ SỞ HỮU (đồng)']
initial_data_set['out_share'] = latest_time_of_raw_data['Số CP lưu hành (Triệu CP)']
initial_data_set['value'] = latest_time_of_intermediary_data['value']
initial_data_set['debt'] = latest_time_of_intermediary_data['debt']
initial_data_set['cash'] = latest_time_of_raw_data['Tiền và tương đương tiền (đồng)']

In [ ]:
raw_data = pd.read_csv('financial_statements.csv')
raw_data['time']= pd.to_datetime(raw_data['time'])
raw_data.set_index(['ticket','time'],inplace=True)
raw_data = raw_data.sort_index(level='time',ascending=True)
raw_data =raw_data[raw_data.index.get_level_values(level='time')>=pd.to_datetime('2020-12-31')]

In [ ]:
#Du lieu cho ky so sanh
initial_data_set['out_share'] = raw_data['Số CP lưu hành (Triệu CP)'].groupby(level=0).last()
deterministic_complementary = pd.DataFrame(raw_data['LỢI ÍCH CỦA CỔ ĐÔNG THIỂU SỐ'
                                                    ]/raw_data['VỐN CHỦ SỞ HỮU (đồng)'],columns=['minor_rate'])
deterministic_complementary['net_debt'] = raw_data['Vay và nợ thuê tài chính ngắn hạn (đồng)'
                                                   ]+raw_data['Vay và nợ thuê tài chính dài hạn (đồng)'
                                                              ]- raw_data['Tiền và tương đương tiền (đồng)']
deterministic_complementary = deterministic_complementary.join(initial_data_set['out_share'],on='ticket')
deterministic_complementary['net_debt_per_share'] = deterministic_complementary['net_debt']/deterministic_complementary['out_share']
deterministic_complementary.drop(labels=['out_share','net_debt'],axis=1,inplace=True)

In [ ]:
share = initial_data_set[['out_share']]
generated_point = intermediary_data[['dttn','wc','value']]
generated_point = generated_point.join(share, on=generated_point.index.names[0])
generated_point['dttn']/=generated_point['out_share']
generated_point['wc']/=generated_point['out_share']
generated_point['value']/=generated_point['out_share']
generated_point.drop('out_share',inplace=True,axis=1)


In [ ]:
#Bo chi tieu can quan tam
input_ratio = np.log(intermediary_data['dttn']/intermediary_data['dttn'].groupby(level='ticket').shift())
input_ratio = input_ratio.to_frame(name='g_log')
input_ratio['omr'] = intermediary_data['ebit']/intermediary_data['dttn']
input_ratio['dpoc'] = intermediary_data['dna']/(intermediary_data['dttn']-intermediary_data['ebit'])
input_ratio['lrr'] = intermediary_data['dna']/intermediary_data['capex']
input_ratio['wct'] = intermediary_data['dttn']/intermediary_data['wc']
input_ratio['hskd'] = intermediary_data['dttn']/intermediary_data['TA']
input_ratio['ht'] = intermediary_data['equity']/intermediary_data['TA']
input_ratio['wod'] = intermediary_data['debt']/(intermediary_data['TA']-intermediary_data['equity']) 
input_ratio = input_ratio[input_ratio.index.get_level_values(level=1).year < 2021]

<h2>2. Xay dung cau truc phu thuoc giua cac bien<h2>

<h3>2.1. Chuan hoa bo chi tieu<h3>

<h4>2.1.1. Cac ham lien quan<h4>

In [ ]:
#Xay dung ham phan phoi dua tren gaussian kde
data_type = Literal['in_R','minus_infty_to_one','binary','zero_to_infty']
infinitesimal = 1e-9
quantity_for_intepld = 1000000
class kde_distribution:
    def __init__(self,data_set,type_of_data:data_type):
        self.type = type_of_data
        data = data_set.replace([np.inf,-np.inf],np.nan)
        data = data_set.dropna()
        if self.type == 'in_R':
            self.data = data
            self.range = np.linspace(-200,200,quantity_for_intepld)
        elif self.type == 'minus_infty_to_one':
            self.data = -np.log(1-data)
            self.range = np.linspace(-200,1-infinitesimal,quantity_for_intepld)
        elif self.type == 'binary':
            data += infinitesimal
            self.data = np.log(data/(1-data))
            self.range = np.linspace(0+infinitesimal,1-infinitesimal,quantity_for_intepld)
        elif self.type == 'zero_to_infty':
            self.data = np.log(data)
            self.range = np.linspace(0+infinitesimal,200,quantity_for_intepld)
        self.kde = stats.gaussian_kde(self.data)
        self._pdf = self.pdf(self.range)
        self.cdf_val = np.cumsum(self._pdf)
        self.cdf_val/= self.cdf_val[-1]
        self._cdf = interp1d(self.range,self.cdf_val,kind='linear',bounds_error=False,fill_value=(0,1))
        self._ppf = interp1d(self.cdf_val,self.range,kind='linear',bounds_error=False, fill_value="extrapolate")
    def pdf(self, x):
        if self.type == 'in_R':
           p = self.kde(x)
        elif self.type == 'minus_infty_to_one':
            p = self.kde(-np.log(1-x))/np.log(1-x) 
        elif self.type == 'binary':
            p = self.kde(np.log(x/(1-x)))/(x-x**2)
        elif self.type == 'zero_to_infty':
            p = self.kde(np.log(x))/x
        return p
    def cdf (self,x):
        return self._cdf(x)
    def ppf (self,u):
        return self._ppf(u)

<h4>2.1.2. Thuat toan normalize chi tieu bang CDF<h4>

In [ ]:
#Phan nhom bo chi tieu
in_R = ['g_log','lrr','wct']
minus_infty_to_one = ['omr','dpoc']
binary = ['ht','wod']
zero_to_infty = ['hskd']
input_ratio = input_ratio.replace([-np.inf,np.inf],np.nan)
ticket = input_ratio.index.get_level_values(level=0).unique()
ratio = input_ratio.columns
standardised_input_ratio = pd.DataFrame()
#Chuan hoa du lieu
for i in ticket:
    data_set = input_ratio.xs(i,level=0)
    single_stock = pd.DataFrame()
    for j in ratio:
        n = len(data_set[j].dropna())+1
        if len(single_stock) == 0:
            single_stock = data_set[[j]].rank(method='average')/n
        else:
            mediate = data_set[[j]].rank(method='average')/n
            single_stock = pd.concat([single_stock,mediate],axis=1)
    single_stock['ticket'] = i
    if len(standardised_input_ratio)==0:
        standardised_input_ratio = single_stock
    else:
        standardised_input_ratio = pd.concat([standardised_input_ratio,single_stock],axis=0)
standardised_input_ratio.set_index('ticket',append=True,inplace=True)
standardised_input_ratio = standardised_input_ratio.reorder_levels(['ticket','time'],axis=0)
    

<h3>2.2. Xay dung thu tu phu thuoc<h3>

<h4>2.2.1. Su phu thuoc giua cac doanh nghiep voi nhau<h4>

In [ ]:
#Ma tran MI cua g_log giua cac doanh nghiep voi nhau
standardised_g_log = standardised_input_ratio[['g_log']].unstack('ticket').droplevel(level=0,axis=1)
outter_correlation_matrix = standardised_g_log.corr(method='kendall')
""" ma trận xếp hạng hệ số tương quan sẽ được xếp giữa trên độ lớn của hệ số tương quan kendall tau(tức là trị tuyết đối)
    Xếp hạng được xếp theo dòng và phân theo thứ tự giảm dần tức là hệ số có tương quan cao nhất sẽ được xếp hạng 1"""
outter_rank_matrix = outter_correlation_matrix.abs().rank(axis=1,ascending=False,method='min')

In [ ]:
outter_dependence_tree = []
stock = 'AAA'
init_stock = [stock]
ticket = list(outter_rank_matrix.columns)
ticket.remove(stock)
while len(ticket) != 0:
    mediate_term = []
    for i in init_stock:
        data = outter_correlation_matrix[i].loc[ticket].sort_values(ascending=True)
        most_correlated = data.iloc[0]
        if most_correlated==2:
            dependence_stock = list(data[data==most_correlated].index)
        else:
            dependence_stock = [data[data==most_correlated].index[0]]
        for j in dependence_stock:
            outter_dependence_tree.append(f'{i}-{j}')
            ticket.remove(j)
            mediate_term.append(j)
    init_stock = mediate_term

<h4>2.2.2. Su phu thuoc trong doanh nghiep<h4>

In [ ]:
ticket = list(standardised_input_ratio.index.get_level_values(level='ticket').unique())
inner_dependence_tree = {}
init_ratio = 'g_log'
for i in ticket:
    sub_independence_tree = []
    data = standardised_input_ratio.xs(i, level='ticket')
    ratio = list(data.columns)
    init_set = [init_ratio]
    ratio.remove(init_ratio)
    corr_matrix = data.corr(method='kendall')
    rank_matrix = corr_matrix.abs().rank(axis=1,ascending=False,method='min')
    while len(ratio) != 0 :
        mediate_term = []
        for j in init_set:
            if len(ratio)==0:
                break
            rank_of_interest = rank_matrix[j].loc[ratio].sort_values(ascending=True)
            most_correlated = rank_of_interest.iloc[0]
            if most_correlated == 2:
                dependence_set = list(rank_of_interest[rank_of_interest==2].index)
            else:
                dependence_set = [rank_of_interest.index[0]]
            for t in dependence_set:
                sub_independence_tree.append(f'{j}-{t}')
                ratio.remove(t)
                mediate_term.append(t)
        init_set = mediate_term
    inner_dependence_tree[i] = sub_independence_tree

<h3>2.3. Xay dung cau truc phu thuoc<h3>

In [ ]:
controls = FitControlsBicop(selection_criterion='aic')

In [ ]:
def bootstrap(data:np.ndarray, sample_size:int, simulation_size:int):
    """Bootstrapping function

    Args:
        data (np.ndarray) shape=(n,d)
        sample_size (int)
        simulation_size (int): how many times bootstrapping
    
    Return:
    ------------
        data (np.ndarray) shape(simulation_size, sample_size, d)
    """
    n = len(data)-1 #vi tri diem cuoi cua data
    position = np.random.randint(0,n,size=(simulation_size,sample_size))
    return data[position]
def empirical_cdf (data:np.ndarray):
    """Empirical cdf used empirical data as the proxy for true cdf which is P(X <= x)

    Args:
        data (np.ndarray): shape = (n,d)
        x (np.ndarray): shape = (n,1,d)
        X (np.ndarray): shape = (1,n,d)
    
    Return
    -------
        A float number lies in range (0,1)^n \in R^n
        
    """
    x = data[:,None,:]
    X = data[None,:,:]
    return np.mean(np.all(X<=x,axis=2),axis=0)
class Cvm_test:
    def __init__(self,cop_func):
        self.copula = cop_func
    def CvM_statistic(self,data:np.ndarray):
        """Cramer - Von Mises statistic
            Formula:
            \sum_{i=1}^n{(C_n(u_i)-C_{\theta}(u_i))^2}
        Args:
            data (np.ndarray): 

        Returns:
            _type_: _description_
        """
        C_n = empirical_cdf(data)
        C_theta = self.copula.cdf(data)
        return np.sum((C_n-C_theta)**2)
    def pvalue(self,data:np.ndarray,number_of_simulation:int,sample_size:int):
        """Pvalue of Cramer - Von Mises statistic

        Args:
            data (np.ndarray): shape = (n,d)
            number_of_simulation (int): 
            sample_size (int): 

        Returns:
            a number lies in (0,1)
        """
        bootstrap_data = bootstrap(data,simulation_size=number_of_simulation,sample_size=sample_size)
        statistic = self.CvM_statistic(data)
        CvM_stats_vector = []
        CvM_stats_vector = np.array([self.CvM_statistic(i) for i in bootstrap_data])
        return np.mean(CvM_stats_vector>=statistic)    

<h4>2.3.1. Su phu thuoc giua cac doanh nghiep voi nhau<h4>

In [ ]:
outter_dependence_structure = {}
number_of_bootsrapping = 10000
ticker = list(standardised_input_ratio.index.get_level_values(level='ticket').unique())
CvM_pvalue_matrix = np.full(shape=(len(ticker),len(ticker)),fill_value=np.nan)
CvM_pvalue_matrix = pd.DataFrame(CvM_pvalue_matrix,index=ticker,columns=ticker)
for pair in outter_dependence_tree:
    pair_name = pair.split('-')
    data = standardised_g_log[pair_name].dropna()
    uv = data.to_numpy()
    cop_func = Bicop()
    cop_func.select(uv,controls=controls)
    # test = Cvm_test(cop_func)
    # pvalue = test.pvalue(uv, number_of_simulation=number_of_bootsrapping,sample_size=uv.shape[0])
    # CvM_pvalue_matrix.loc[pair_name[0],pair_name[1]] = pvalue
    outter_dependence_structure[pair] = cop_func

<h4>2.3.2. Su phu thuoc ben trong doanh nghiep(giua cac chi tieu voi nhau)<h4>

In [ ]:
ticker = list(standardised_input_ratio.index.get_level_values(level='ticket').unique())
list_of_ratio = standardised_input_ratio.columns.to_list()
inner_cvm_pvalue = pd.DataFrame()
number_of_bootsrapping = 10000
inner_dependence_structure = {}
n = len(list_of_ratio)
for i in ticker:
    sub_pvalue_matrix = np.full(shape=(n,n),fill_value=np.nan)
    sub_pvalue_matrix = pd.DataFrame(sub_pvalue_matrix,index=list_of_ratio,columns=list_of_ratio)
    sub_inner_dependence_tree = inner_dependence_tree[i]
    data = standardised_input_ratio.xs(i,level='ticket')
    single_stock = {}
    for pair in sub_independence_tree:
        ratio_pair = pair.split('-')
        uv = data[ratio_pair].dropna().to_numpy()
        cop_func = Bicop()
        cop_func.select(uv,controls=controls)
        single_stock[pair] = cop_func
        # test = Cvm_test(cop_func)
        # pvalue = test.pvalue(uv,number_of_simulation=number_of_bootsrapping,sample_size=uv.shape[0])
        # sub_pvalue_matrix.loc[ratio_pair[0],ratio_pair[1]] = pvalue
    # sub_pvalue_matrix['ticket'] = i
    inner_dependence_structure[i] = single_stock
#     if len(inner_cvm_pvalue)==0:
#         inner_cvm_pvalue = sub_pvalue_matrix
#     else:
#         inner_cvm_pvalue = pd.concat([inner_cvm_pvalue,sub_pvalue_matrix],axis=0)
# inner_cvm_pvalue.index.names = ['ratio']
# inner_cvm_pvalue.set_index('ticket',append=True,inplace=True)
# inner_cvm_pvalue = inner_cvm_pvalue.reorder_levels(['ticket','ratio'])

<h2>3. Xay dung phan phoi xac suat cho ky so sanh<h2>

<h3>3.1. Xay dung cac ham can su dung<h3>

In [ ]:
in_R = ['g_log','lrr','wct']
minus_infty_to_one = ['omr']
binary = ['ht','wod','dpoc']
zero_to_infty = ['hskd']
data_type = Literal['in_R','minus_infty_to_one','binary','zero_to_infty']
def objective_function_theta(parameter_vector:np.ndarray, q_vector:np.ndarray, projection_vector:np.ndarray, type: data_type,infinitesimal:float  ):
    """Objective function is the function that have to be optimized to estimate parameter for projected distribution

    Args:
        parameter_vector (np.ndarray): [alpha, beta]
        q_vector (np.ndarray): shape = (n,) represent quantile position of projected value
        projection_vector (np.ndarray): shape = (n,) the vector of projected value
        infinitestimal (float): use when there is overlap in projection vector 
    Return
    -------
        Output is a real number
    """
    if type == 'binary':
        data_vector = projection_vector
    elif type == 'in_R':
        data_vector = 1/(1+np.exp(-projection_vector))
    elif type == 'minus_infty_to_one':
        data_vector = 2/(1+np.exp(1-projection_vector))
    elif type == 'zero_to_infty':
        data_vector = projection_vector/(1+projection_vector)
    if len(np.unique(data_vector)) == 1:
        adjust_lower = np.array([i*infinitesimal for i in range(0,len(data_vector))])
        adjust_upper = adjust_lower[::-1]
        boon_type = abs(0-data_vector[0]) <= abs(1-data_vector[-1])
        data_vector = data_vector + boon_type*adjust_lower - (1-boon_type)*adjust_upper
    alpha, beta = parameter_vector
    quantile_vector = stats.beta.cdf(data_vector,a = alpha, b = beta)
    deviation_vector = q_vector - quantile_vector
    return deviation_vector.T@deviation_vector
def optimize_function (q_vector:np.ndarray, projection_vector:np.ndarray, type: data_type,infinitesimal:float):
    """Objective function is the function that have to be optimized to estimate parameter for projected distribution

    Args:
        q_vector (np.ndarray): shape = (n,) represent quantile position of projected value
        projection_vector (np.ndarray): shape = (n,) the vector of projected value
        infinitestimal (float): use when there is overlap in projection vector 
        
    Return
    -------
        A np.ndarray of alpha and beta
    """
    bounds = [(1e-9,1e9),(1e-9,1e9)]
    init = [0.1,0.1]
    res = minimize(objective_function_theta,init,args=(q_vector,projection_vector,type,infinitesimal),bounds=bounds)
    return res.x
class projected_distribution:
    def __init__(self,q_vector:np.ndarray,projected_vector:np.ndarray, data_type:data_type,infinitesimal:float):
        self.type = data_type
        self.alpha, self.beta = optimize_function(q_vector,projected_vector,self.type,infinitesimal)
        self.loss = objective_function_theta(np.array([self.alpha,self.beta]),q_vector,projected_vector,self.type,infinitesimal)
    def pdf (self, x):
        if self.type == 'binary':
            x_trans = x
            pdf_val = stats.beta.pdf(x_trans,a=self.alpha,b=self.beta)
        elif self.type == 'in_R':
            x_trans = 1/(1+np.exp(-x))
            pdf_val = stats.beta.pdf(x_trans,a=self.alpha,b=self.beta)*np.exp(-x)/(1+np.exp(-x))**2
        elif self.type == 'minus_infty_to_one':
            x_trans = 2/(1+np.exp(1-x))
            pdf_val = stats.beta.pdf(x_trans,a=self.alpha,b=self.beta)*2*np.exp(1-x)/(1+np.exp(1-x))**2
        elif self.type == 'zero_to_infty':
            x_trans = x/(1+x)
            pdf_val = stats.beta.pdf(x_trans,a=self.alpha,b=self.beta)/(1+x)**2
        return pdf_val
    def cdf (self, x):
        if self.type == 'binary':
            x_trans = x 
        elif self.type == 'in_R':
            x_trans = 1/(1+np.exp(-x))
        elif self.type == 'minus_infty_to_one':
            x_trans = 2/(1+np.exp(1-x))
        elif self.type == 'zero_to_infty':
            x_trans = x/(1+x)
        cdf_val = stats.beta.cdf(x_trans,a=self.alpha,b=self.beta)  
        return cdf_val
    def ppf (self, u,lower_bounds,upper_bounds):
        x_trans = stats.beta.ppf(u,a=self.alpha,b=self.beta)
        if self.type == 'binary':
            ppf_val = x_trans
        elif self.type == 'in_R':
            ppf_val = np.log(x_trans/(1-x_trans))
            ppf_val = np.clip(ppf_val,lower_bounds,upper_bounds)
        elif self.type == 'minus_infty_to_one':
            ppf_val = np.log(x_trans/(2-x_trans))+1
            ppf_val = np.clip(ppf_val,-lower_bounds,1)
        elif self.type == 'zero_to_infty':
           ppf_val = x_trans/(1-x_trans)
           ppf_val = np.clip(ppf_val,1e-6,upper_bounds)
        return ppf_val
    def loss (self):
        return self.loss

<h3>3.2. Thuat toan<h3>

In [ ]:
#Lay du lieu
input_data = pd.read_csv('ratio_data.csv')
input_data['time'] = pd.to_datetime(input_data['time'])
input_data = input_data.sort_values('time',ascending=True)
input_data.set_index(['ticket','time'],inplace=True)
input_data = input_data[input_data.index.get_level_values(level='time').year>2020]

In [ ]:
#Dau vao de xac dinh phan phoi xac suat
adjusted_range = 1
ticker = list(standardised_input_ratio.index.get_level_values(level='ticket').unique())
q_vector = np.array([0.05,0.25,0.5,0.75,0.95])
data_for_distribution = pd.DataFrame()
for i in q_vector:
    name = f'Q{i}'
    q = input_data.groupby(level='ticket').quantile(i)
    q['Q'] = name
    q.set_index('Q',inplace=True, append=True)
    q = q.unstack(level=1).stack(level=0)
    if len(data_for_distribution) == 0:
        data_for_distribution = q
    else:
        data_for_distribution = pd.concat([data_for_distribution,q],axis=1)

lower = input_data.groupby(level="ticket").min()
lower['range'] ='L'
lower.set_index('range',inplace=True,append=True)
lower = lower.unstack(level=1).stack(level=0)
lower[lower>0]/=adjusted_range
lower[lower<=0]*=adjusted_range

upper = input_data.groupby(level="ticket").max()
upper['range'] ='U'
upper.set_index('range',inplace=True,append=True)
upper = upper.unstack(level=1).stack(level=0)
upper[upper>0]*adjusted_range
upper[upper<=0]/=adjusted_range
range_input = pd.concat([lower,upper],axis=1)

In [ ]:
#Luu tru phan phoi xac xuat
in_R = ['g_log','lrr','wct']
minus_infty_to_one = ['omr']
binary = ['ht','wod','dpoc']
zero_to_infty = ['hskd']
infinitesimal = 1e-9
ticker = list(standardised_input_ratio.index.get_level_values(level='ticket').unique())
projected_distribution_dict = {}
for i in ticker:
    data = data_for_distribution.xs(i,level='ticket')
    store_dict = {}
    ind = data.index.to_list()
    for j in ind:
        projected_vector = data.loc[j].to_numpy()
        if j in in_R:
            func = projected_distribution(q_vector=q_vector,
                                          projected_vector=projected_vector,
                                          data_type='in_R',infinitesimal=infinitesimal)
        elif j in minus_infty_to_one:
            func = projected_distribution(q_vector=q_vector,
                                          projected_vector=projected_vector,
                                          data_type='minus_infty_to_one',infinitesimal=infinitesimal)
        elif j in binary:
            func = projected_distribution(q_vector=q_vector,
                                          projected_vector=projected_vector,
                                          data_type='binary',infinitesimal=infinitesimal)
        elif j in zero_to_infty:
            func = projected_distribution(q_vector=q_vector,
                                          projected_vector=projected_vector,
                                          data_type='zero_to_infty',infinitesimal=infinitesimal)
        
        store_dict[j] = func
    projected_distribution_dict[i] = store_dict                     


<h2>4. Cac tham so cho Wacc(chi phi su dung von binh quan)<h2>

<h3>4.1. Chi phi su dung von chu <h3>

In [ ]:
#Du lieu ve market_portfolio
rm_data = pd.read_csv('VNI_price.csv')
rm_data['time'] = pd.to_datetime(rm_data['time'])
rm_data.set_index('time',inplace=True)
rm_data = rm_data[(rm_data.index.year<2021)&(rm_data.index.year > 2015)]
rm_data = rm_data.pct_change()
#du lieu ve cac co phieu
stock_return = pd.read_csv('input_data.csv')
stock_return['time'] = pd.to_datetime(stock_return['time'])
stock_return.set_index('time',inplace=True)
stock_return = stock_return[(stock_return.index.year < 2021) & (stock_return.index.year > 2015)]
stock_return = stock_return.pct_change()

In [ ]:
#Tinh beta
beta_vector = []
beta_dict = {}
ticker = list(standardised_input_ratio.index.get_level_values(level='ticket').unique())
for i in ticker:
    y = stock_return[i]
    uv = pd.concat([rm_data,y],axis=1)
    uv.dropna(inplace=True)
    covariance_matrix = uv.cov()
    beta = covariance_matrix.iloc[0,1]/covariance_matrix.iloc[0,0]
    beta_vector.append(beta)
    beta_dict[i] = beta
beta_vector = np.array(beta_vector)

In [ ]:
#cac tham so khac
rf = pd.read_csv('interest_rate_10_year_goverment_bond.csv')
rf['time'] = pd.to_datetime(rf['time'])
rf.set_index('time', inplace=True)
annual_estimated_rf = rf[rf.index.year > 2020].mean().values/100

rm = pd.read_csv('VNI_price.csv')
rm['time'] = pd.to_datetime(rm['time'])
rm.set_index('time', inplace=True)
rm = rm.pct_change()
annual_estimated_rm = rm[rm.index.year > 2020].mean().values*12


In [ ]:
annual_re_data = annual_estimated_rf + beta_vector*(annual_estimated_rm-annual_estimated_rf)
annual_re_data = pd.DataFrame(annual_re_data,index=ticker,columns=['re'])

<h4>4.2. Chi phi su dung no binh quan<h4>

In [ ]:
debt_related_data = pd.read_csv('intermediary_data.csv')
debt_related_data['time'] = pd.to_datetime(debt_related_data['time'])
debt_related_data.set_index(['ticket','time'],inplace=True)
debt_related_data = debt_related_data.sort_index(level=0,ascending=True)
debt_related_data = debt_related_data[['interest','debt']]
debt_related_data['average_debt'] = debt_related_data['debt'].groupby(level=0).rolling(window=2).mean().droplevel(level=0)
debt_related_data['cod'] = debt_related_data['interest']/debt_related_data['average_debt']
annual_estimated_cod = debt_related_data['cod'].groupby(level=0).mean().replace(np.inf,np.nan)*4
annual_estimated_cod = annual_estimated_cod.replace(np.nan,0)

<h2>5. Mo phong Monte Carlo<h2>

In [ ]:
simulated_size = 10000
simulated_col = np.arange(1,simulated_size+1)
lower_bounds = 0.05
upper_bounds = 0.95
simulated_ind = pd.date_range(start='2021-01-01',end='2026-01-01',freq='1Q')
origin = np.random.uniform(lower_bounds,upper_bounds, size=(len(simulated_ind),simulated_size))

In [ ]:
def conditioned_simulation (u1:np.ndarray, copula_func):
    """ We have the following conditional distribution p(u2|u1) and we want to simulate u2 by
    this conditional distribution

    Args:
        u1 (np.ndarray): observed data
        copula_func (_type_): act as the activation conditional distribution
        
    Return:
    An np.ndarray have the same shape as u1
    """
    shape = u1.shape
    processed_u1 = u1.reshape(-1,1)
    prob = np.random.uniform(lower_bounds,upper_bounds,size=processed_u1.shape)
    u = np.column_stack([prob,processed_u1])
    u2 = cop_func.hinv2(u)
    return u2.reshape(shape[0],shape[1])

<h3>5.1. chi tieu g_log cua cac doanh nghiep trong khung chuan hoa<h3>

In [ ]:
uni_simulated_glog_dict = {}
for i in outter_dependence_structure.keys():
    pair_of_share = i.split('-')
    s1, s2 = pair_of_share
    if len(uni_simulated_glog_dict) == 0:
        uni_simulated_glog_dict[s1] = origin
    
    function = outter_dependence_structure[i]
    u1 = uni_simulated_glog_dict[s1]
    u2 = conditioned_simulation(u1,function)
    uni_simulated_glog_dict[s2] = u2


<h3>5.2. Bo chi tieu trong khung chuan hoa<h3>


In [ ]:
ratio_of_interest = pd.read_csv('ratio_data.csv').columns[2::].to_list()
ticker = pd.read_csv('ratio_data.csv')['ticket'].unique().tolist()
uni_simulation_dict = {}
for i in inner_dependence_structure.keys():
    single_stock_dict = {}
    sub_structure = inner_dependence_structure[i]
    for j in sub_structure.keys():
        function = sub_structure[j]
        ratio_1, ratio_2 = j.split('-')
        if len(single_stock_dict)==0:
            single_stock_dict[ratio_1] = uni_simulated_glog_dict[i]
        u1 = single_stock_dict[ratio_1]
        u2 = conditioned_simulation(u1,function)
        single_stock_dict[ratio_2] = u2
    uni_simulation_dict[i] = single_stock_dict
del uni_simulated_glog_dict

<h3>5.3. Gia tri du bao cua bo chi tieu<h3>

In [ ]:
simulation_dict = {}
for i in uni_simulation_dict.keys():
    sub_data = uni_simulation_dict[i]
    function_set = projected_distribution_dict[i]
    bound = range_input.xs(i,level=0)
    sub_simulation = {}
    for j in sub_data.keys():
        function = function_set[j]
        u = sub_data[j]
        low = bound.loc[j,'L']
        up = bound.loc[j,'U']
        projected = function.ppf(u,low,up)
        sub_simulation[j] = projected
    simulation_dict[i] = sub_simulation
del uni_simulation_dict

<h2>6. Phan phoi loi suat sinh loi<h2>

<h3>6.1. Cac ham can su dung<h3>

In [ ]:
def present_value_function (cf: np.ndarray, wacc: np.ndarray, v_0: np.ndarray, v_1: np.ndarray):
    """Calculating the continuous cashflow

    Args:
        cf (np.ndarray): matrix represent the simulation of FCFF over time, shape = (t,n)
        wacc (np.ndarray): matrix represent the simulation of wacc overtime, shape = (t,n)
        v_0 (np.ndarray): matrix represent the simulation of firm value at the start of a period, shape = (t,n)
        v_1 (np.ndarray): matrix represent the simulation of firm value at the end of a period, shape = (t,n)
    
    Returns:
        A matrix represent of present value of cashflow at each period, shape = (t,n)
    """
    one_matrix = np.full(shape=wacc.shape,fill_value=1)
    cum_wacc = np.cumsum(wacc,axis=0)
    delta_v = v_1 - v_0
    average_v = (v_0+v_1)/2
    first_term = 1/(wacc * np.exp(cum_wacc))
    second_term = v_0*(np.exp(wacc) - one_matrix) + delta_v*(np.exp(wacc) -wacc - 1)/wacc
    third_term = cf/average_v
    return first_term*second_term*third_term
def future_value_function (cf: np.ndarray, wacc: np.ndarray, v_0: np.ndarray, v_1: np.ndarray):
    """Calculating the continuous cashflow

    Args:
        cf (np.ndarray): matrix represent the simulation of FCFF over time, shape = (t,n)
        wacc (np.ndarray): matrix represent the simulation of wacc overtime, shape = (t,n)
        v_0 (np.ndarray): matrix represent the simulation of firm value at the start of a period, shape = (t,n)
        v_1 (np.ndarray): matrix represent the simulation of firm value at the end of a period, shape = (t,n)
    
    
    Returns:
        A matrix represent of future value of cashflow at each period, shape = (t,n)
    """
    time_length = wacc.shape[0]
    present_value = present_value_function(cf,wacc,v_0,v_1)
    cum_wacc = np.sum(wacc,axis=0)
    adjust_term = np.tile(cum_wacc,(time_length,1))
    return present_value*np.exp(adjust_term)


In [ ]:
def positive_condition (data:np.ndarray):
    """working like if else function

    Returns:
    An array has the same shape as data in which:
    If x > 0: return 1
    If x < 0: return 0
    """
    sign_matrix = np.sign(data)
    return abs((sign_matrix+1)/2)
def negative_condition (data:np.ndarray):
    """working like if else function

    Returns:
    An array has the same shape as data in which
    If x < 0: return 1
    If x > 0: return 0
    """   
    one_matrix = np.full(shape=data.shape, fill_value=1)
    sign_matrix = np.sign(data)
    return -(sign_matrix-one_matrix)/2

In [ ]:
def EV_calculation (cf: np.ndarray, wacc: np.ndarray, v_0: np.ndarray, v_1: np.ndarray, r_inf: float):
    """Calculating enterprise value

    Args:
        cf (np.ndarray): matrix represent the simulation of FCFF over time, shape = (t,n)
        wacc (np.ndarray): matrix represent the simulation of wacc overtime, shape = (t,n)
        v_0 (np.ndarray): matrix represent the simulation of firm value at the start of a period, shape = (t,n)
        v_1 (np.ndarray): matrix represent the simulation of firm value at the end of a period, shape = (t,n)
        r_inf (float): inflation rate
    
    Returns:
        A vector of EV
    """
    present_value = present_value_function(cf,wacc,v_0,v_1)
    postive_present_value = present_value_function(cf,wacc,v_0,v_1)*positive_condition(present_value)
    tpv = np.sum(postive_present_value,axis=0)
    cum_wacc_vector = np.sum(wacc,axis=0)
    r_inf_matrix = r_inf*np.full(shape=cf.shape,fill_value=1)
    total_rinf = np.sum(r_inf_matrix,axis=0)
    first_term = tpv*np.exp(cum_wacc_vector)
    second_term = np.exp(cum_wacc_vector)-np.exp(total_rinf*positive_condition(np.exp(cum_wacc_vector)-np.exp(total_rinf)))
    return first_term/second_term
def hidden_cost_function (cf: np.ndarray, wacc: np.ndarray, v_0:np.ndarray, v_1: np.ndarray):
    """Calculating hidden cost

    Args:
        cf (np.ndarray): matrix represent the simulation of FCFF over time, shape = (t,n)
        wacc (np.ndarray): matrix represent the simulation of wacc overtime, shape = (t,n)
        v_0 (np.ndarray): matrix represent the simulation of firm value at the start of a period, shape = (t,n)
        v_1 (np.ndarray): matrix represent the simulation of firm value at the end of a period, shape = (t,n)
    
    Returns:
        A vector of hidden cost
    """ 
    present_value = present_value_function(cf,wacc,v_0,v_1)
    negative_present_value = present_value*negative_condition(present_value)
    return -np.sum(negative_present_value,axis=0)

In [ ]:
def benefit_function (cf: np.ndarray, wacc: np.ndarray, v_0: np.ndarray, v_1: np.ndarray, r_inf: float):
    """Calculating the numerator in return formula

    Args:
        cf (np.ndarray): matrix represent the simulation of FCFF over time, shape = (t,n)
        wacc (np.ndarray): matrix represent the simulation of wacc overtime, shape = (t,n)
        v_0 (np.ndarray): The latest reported v matrix
        v_1 (np.ndarray): projected v matrix
        r_inf (float): inflation rate
    
    Returns:
        A matrix of benefit
    """
    EV_0 = EV_calculation(cf,wacc,v_0,v_1,r_inf)
    future_value = future_value_function(cf,wacc,v_0,v_1)
    positive_future_value = future_value*positive_condition(future_value)
    tfv = np.sum(positive_future_value,axis=0)
    return tfv + EV_0 * np.exp(r_inf*cf.shape[0])
def projected_return (cf: np.ndarray, wacc: np.ndarray, v_0: np.ndarray, v_1: np.ndarray, r_inf: float,current_market_price:float,net_debt:np.ndarray,minority:float,period_per_year:float):
    """Estimating the distribution of stock return

    Args:
        cf (np.ndarray): matrix represent the simulation of FCFF over time, shape = (t,n)
        wacc (np.ndarray): matrix represent the simulation of wacc overtime, shape = (t,n)
        v_0 (np.ndarray): matrix represent the simulation of firm value at the start of a period, shape = (t,n)
        v_1 (np.ndarray): matrix represent the simulation of firm value at the end of a period, shape = (t,n)
        r_inf (float): annual inflation rate
        current_market_price (float): market price
        net_debt (np.ndarray): debt - cash
        minority (float): ratio of minority shareholders and total shareholders
        period_per_year (float): number of period per year
    """
    benefit = benefit_function(cf,wacc,v_0,v_1,r_inf)*(1-minority)
    cost = current_market_price + (hidden_cost_function(cf,wacc,v_0,v_1) + net_debt)*(1-minority)
    hpr = benefit/cost
    number_of_period = cf.shape[0]
    number_of_year = number_of_period/period_per_year
    return hpr**(1/number_of_year)-1
    

In [ ]:
def input_set_generate (ticker: list, generator: pd.DataFrame, simulation_ratio: dict, 
                        current_date, tax_rate: float,annual_re:pd.DataFrame, annual_cod:pd.DataFrame):
    """Generating dictionary of FCFF and dictionary of wacc and value

    Args:
        ticker (list): the list of stock
        generator (pd.DataFrame): the latest reported dttn, working capital and value
        simulation_ratio (dict): dictionary of simulation ratio
        current_date (_type_): 
        tax_rate (float): corporate level of tax
        annual_re (pd.DataFrame): contains ownerships required rate of return of each stock in ticker list
        annual_cod (pd.DataFrame): contains cost of using debt of each stock in ticker list

    Returns:
        (list): contain FCFF dictionary and dictionary of wacc and value
    """
    latest_reported_date = current_date - pd.offsets.QuarterEnd()
    gen_data = generator.xs(latest_reported_date,level='time')
    output = []
    FCFF = {}
    complementary_dict = {}
    for i in ticker:
        complementary = {}
        simulation_set = simulation_ratio[i]
        dttn_0, wc_0,v_0 = gen_data.loc[i].tolist()
        cummulative_glog = np.cumsum(simulation_set['g_log'],axis=0)
        dttn = dttn_0*np.exp(cummulative_glog)
        ebit = dttn*simulation_set['omr']
        dna = (dttn - ebit)*simulation_set['dpoc']
        capex = dna*simulation_set['lrr']
    
        wct = np.maximum(np.abs(simulation_set['wct']),0.1)*np.sign(simulation_set['wct'])
        wc = dttn/wct
        wc_0 = np.full(shape=(1,dttn.shape[1]),fill_value=wc_0)
        full_wc = np.vstack([wc_0,wc])
        delta_wc = np.diff(full_wc,axis=0)
    
        cf = ebit*(1-tax_rate) + dna  - capex - delta_wc
        FCFF[i] = cf
    
        equity = dttn*simulation_set['ht']/simulation_set['hskd']
        debt = (1-simulation_set['ht'])/simulation_set['ht']*equity*simulation_set['wod']
        value = equity + debt
        epv = equity/value
        initial_value = np.full(shape=(1,value.shape[1]),fill_value=v_0)
        value = np.vstack([initial_value,value])
    
        re = annual_re['re'].loc[i]
        cod = annual_cod.loc[i]
        wacc = epv*re + (1-epv)*cod*(1-tax_rate)
    
        complementary['value'] = value
        complementary['wacc'] = wacc

        complementary_dict[i] = complementary
    [FCFF, complementary_dict]  
    return [FCFF, complementary_dict] 
        

In [ ]:
period_selection = Literal['annual','quarter']
def return_matrix_construction (ticker: list, cf_dict:dict,complementary: dict, 
                                deterministic_complementary:pd.DataFrame, market_price:pd.DataFrame,r_inf:float, current_date , period:period_selection):
    """Objective of this function is constructing a matrix with columns represent the simulation of a stock and row is number or stock

    Args:
        ticker (list): list of interested stock
        cf_dict (dict): This dict store information of FCFF simulation of each stock
        complementary (dict): This is stochastic complementary: wacc and value simulation,keys = ["wacc","value"]
        deterministic_complementary (pd.DataFrame): proportion of minority and net debt per share, it should be a multiindex data
        market_price (pd.Dataframe): a vector represent the current market price of interested stock
        r_inf (float): inflation rate
        start_date (_type_): the current day
        period (period_selection): annual or quarter
    """
    market_price = market_price[ticker]
    date = current_date - pd.offsets.MonthEnd()*(1-current_date.is_month_end)
    latest_reported_date = current_date - pd.offsets.QuarterEnd()
    deterministic_complementary = deterministic_complementary.xs(latest_reported_date,level='time')
    price_vector = market_price.loc[date]
    if period == "annual":
        period_per_year = 1
    elif period == "quarter":
        period_per_year = 4
    matrix = []
    for i in ticker:
        cf = cf_dict[i]
        wacc = complementary[i]['wacc']
        value = complementary[i]['value']
        v_0 = value[:-1]
        v_1 = value [1::]
        minor_rate, net_debt = deterministic_complementary.loc[i].tolist()
        price = price_vector.loc[i]*1000
        r = projected_return(cf,wacc,v_0,v_1,r_inf/period_per_year,price,net_debt,minor_rate,period_per_year )
        if len(matrix) == 0:
            matrix = r
        else:
            matrix = np.column_stack([matrix,r])
    return matrix

<h3>6.2. Loi suat sinh loi cua tai san<h3>

In [ ]:
current_date = pd.to_datetime('2021-01-01')
market_price = pd.read_csv('compare_data.csv')
market_price['time'] = pd.to_datetime(market_price['time'])
market_price.set_index('time',inplace=True)
t = market_price.loc[['2021-01-04']]
t.index = pd.to_datetime(['2020-12-31'])
market_price = pd.concat([t,market_price])
mp_data = market_price.resample('1M').last()

ticker = list(mp_data.columns)
tax_rate = 0.2
r_inf = 0.03
FCFF_dict, complementary = input_set_generate(ticker,generated_point,simulation_dict,current_date,tax_rate,annual_re_data,annual_estimated_cod)
simulation_return = return_matrix_construction(ticker,FCFF_dict,complementary,deterministic_complementary,mp_data,r_inf,current_date,"quarter")

<h2>7. Phan bo danh muc<h2>

<h3>7.1. Cac ham can dung<h3>

In [ ]:
def CoR_calculation (r: np.ndarray, re:float):
    """calculating the risk measure(Cost of Risk-CoR)

    Args:
        r (np.ndarray): vector of return simulation
        re (float): shareholder's required rate of return ve

    Returns:
        A real number
    """
    L = re - r
    positive_L = L*positive_condition(L)
    return np.mean(positive_L)

def objective_function_1 (theta:np.ndarray, return_matrix:np.ndarray, re_vector:np.ndarray, rf: float):
    """Function used to find the optimal weight

    Args:
        theta (np.ndarray): vector of [w,alpha]
        return_matrix (np.ndarray): shape = (n, number of stock)
        re_vector (np.ndarray): vector of re of each stock
        rf (float): inflation rate

    Returns:
        a real number
    """
    w = theta[:-1]
    alpha = theta[-1]
    re_p = re_vector.T@w
    return_vector = return_matrix@w
    Erp = np.mean(return_vector)
    cor_p = CoR_calculation(return_vector,re_p)
    first_term = alpha*np.log(Erp+1)
    second_term = (1-alpha)*(np.log(1/(1-cor_p))-np.log(1+rf))
    third_term = np.log(alpha)+np.log(1-alpha)
    return -(first_term - second_term + third_term)

def single_stock_utility (r:np.ndarray, re:float, rf:float ):
    Er = np.mean(r)
    cor = CoR_calculation(r,re)
    t = np.log(Er+1)+np.log(1/(1-cor))-np.log(1+rf)
    delta = (t-2)**2 + 4*t
    a1 = ((2-t)-np.sqrt(delta))/(-2*t)
    a2 = ((2-t)+np.sqrt(delta))/(-2*t)
    alpha = max(a1,a2)
    first_term = (Er+1)**alpha
    second_term = ((1-cor)*(1+rf))**(1-alpha)
    return first_term*second_term


In [ ]:
def objective_function_markowitz ( weight:np.ndarray, matrix_return:np.ndarray):
    covariance_matrix = np.cov(matrix_return,rowvar=False)
    var = weight.T@covariance_matrix@weight
    Er = np.mean(matrix_return@weight)
    return - Er/np.sqrt(var)
def markowitz_weight (matrix_return:np.ndarray):
    init = np.full(shape=(matrix_return.shape[1],),fill_value=1)/matrix_return.shape[1]
    bounds = [(0,1) for i in range(matrix_return.shape[1])]
    constrains = {'type':'eq', 'fun': lambda w: np.sum(w)-1}
    w = minimize(objective_function_markowitz,init, args=(matrix_return),bounds=bounds,constraints=constrains)
    return w.x

<h3>7.3. Hold and forget strategy<h3>

In [ ]:
def constrains_func (theta):
    return np.sum(theta[:-1]) -1
constrains = {'type':'eq', 'fun':constrains_func}
def theta_optimize (r_matrix:np.ndarray,re_vector:np.ndarray,rf:float,bounds: list):
    init_theta = np.full(shape=(r_matrix.shape[1]+1,),fill_value=1)/(r_matrix.shape[1]+1)
    w_opt = minimize(objective_function_1,init_theta,args=(r_matrix,re_vector,rf),bounds=bounds,
                    constraints=constrains)
    return w_opt.x

In [ ]:
#phan bo theo mo hinh
bounds = [(0,1) for i in range(simulation_return.shape[1]+1)]
re_vector = annual_re_data.loc[ticker].to_numpy()
proposed_model_w = theta_optimize(simulation_return,re_vector,0.03,bounds)[:-1]
#phan bo deu
naive_w = np.full(shape=(simulation_return.shape[1]),fill_value=1)/simulation_return.shape[1]
#markowitz
markowitz_input = pd.read_csv('input_data.csv')
markowitz_input['time'] = pd.to_datetime(markowitz_input['time'])
markowitz_input.set_index('time',inplace=True)
markowitz_input = markowitz_input[ticker][((markowitz_input.index.year>2015)&(markowitz_input.index.year<2021))]
markowitz_input = np.log(markowitz_input/markowitz_input.shift())
markowitz_input.dropna(inplace=True)
markowitz_w = markowitz_weight(markowitz_input.to_numpy())
#VNIndex
vni = pd.read_csv('VNI_price.csv')
vni['time'] = pd.to_datetime(vni['time'])
vni.set_index('time',inplace=True)
vni = vni[vni.index.year>2020]

In [ ]:
budget = 1e6
ask_price = mp_data[ticker].iloc[0].to_numpy()

proposed_model_postion = budget*proposed_model_w/ask_price
naive_position = budget*naive_w/ask_price
markowitz_position = budget*markowitz_w/ask_price
vni_postion = budget/vni['vni'].iloc[0]

In [ ]:
compare_matrix = mp_data[ticker].to_numpy()
proposed_wealth = compare_matrix@ proposed_model_postion
naive_wealth = compare_matrix@naive_position
markowitz_wealth = compare_matrix@markowitz_position
vni_wealth = vni.to_numpy()*vni_postion

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(proposed_wealth,label='Proposed model 1 performance')
plt.plot(markowitz_wealth,label = 'Markowitz_performance')
plt.plot(naive_wealth,label="Naive performance")
plt.plot(vni_wealth, label='VNIndex performance')
plt.title('Buy and Forget Strategy')
plt.legend()


<h3>7.4. Active management strategy<h3>

In [ ]:
def active_strategy_position (previous_weight: np.ndarray, current_weight: np.ndarray,current_mp: np.ndarray, transaction_fee_rate:float,cash_holding:float, stock_holding: float ):
    """To calculate the current position of portfolio

    Args:
        previous_weight (np.ndarray): The  proportion of each stock in portfolio in the previous period
        current_weight (np.ndarray): The  proportion of each stock in the previous period
        current_mp (np.ndarray): vector of current market price
        transaction_fee_rate (float): the proportion of transaction value investor have pay to settle transactionn
        cash_holding (float): value of cash investor hold
        stock_holding (float): value of stock portfolio

    Returns:
        np.ndarray: a vector of postion of each stock
    """
    w_diff = current_weight - previous_weight
    sell_fee = -w_diff*negative_condition(w_diff)*stock_holding*transaction_fee_rate
    sell_fee = np.sum(sell_fee)
    
    buy_fee = w_diff*positive_condition(w_diff)*(cash_holding+stock_holding)*transaction_fee_rate
    buy_fee = np.sum(buy_fee)
    transaction_fee = sell_fee + buy_fee
    budget = stock_holding + cash_holding - transaction_fee
    position = budget*current_weight/current_mp
    return position
def markowitz_input_function (raw_input:pd.DataFrame, current_date, number_of_month: float):
    """

    Args:
        raw_input (pd.DataFrame): matrix of stock price with row is sample size and columns is stock
        current_date (_type_): 
        number_of_month (float): sample size

    Returns:
        np.ndarray: a matrix of stock log return
    """
    start_date = current_date - pd.offsets.MonthEnd(number_of_month)
    date_range =  pd.date_range(start=start_date,end=current_date,freq='1M')
    marko_input = raw_input.loc[date_range]
    marko_input = np.log(marko_input/marko_input.shift())
    marko_input.dropna(inplace=True)
    return marko_input.to_numpy()

In [ ]:
markowitz_input = pd.read_csv('input_data.csv')
markowitz_input['time'] = pd.to_datetime(markowitz_input['time'])
markowitz_input.set_index('time',inplace=True)
markowitz_input = markowitz_input[ticker]

In [ ]:
bounds = [(0,1) for i in range(simulation_return.shape[1]+1)]
re_vector = annual_re_data.loc[ticker].to_numpy()
fee_rate = 0.005
rf = 0.03
monthly_invested = 1e6
initital_invest= 1e7
previous_proposed_postition = np.full(shape=(simulation_return.shape[1]),fill_value=0)
previous_proposed_weight = np.full(shape=(simulation_return.shape[1]),fill_value=0)
proposed_port_value = []
injected_capital = 0.00
injected_capital_vector = []
proposed_matrix_weight = []

previous_markowitz_position = np.full(shape=(simulation_return.shape[1]),fill_value=0)
previous_markowitz_weight = np.full(shape=(simulation_return.shape[1]),fill_value=0)
markowitz_port_value = []
marko_matrix_weigtht = []
for date in mp_data.index[1::]:
    cash = initital_invest+ monthly_invested
    injected_capital += cash
    injected_capital_vector.append(injected_capital)
    mp = mp_data[ticker].loc[date].to_numpy()  
    #Proposed Model
    
    stock_holding = previous_proposed_postition.T@mp
    FCFF_dict, complementary = input_set_generate(ticker,generated_point,simulation_dict,date,tax_rate,annual_re_data,annual_estimated_cod)
    simulation_return = return_matrix_construction(ticker,FCFF_dict,complementary,deterministic_complementary,mp_data,r_inf,date,"quarter")
    proposed_w = theta_optimize(simulation_return,re_vector,rf,bounds)[:-1]
    proposed_position = active_strategy_position(previous_proposed_weight,proposed_w,mp,fee_rate,cash,stock_holding)

    previous_proposed_postition = proposed_position
    previous_proposed_weight = proposed_w
    #markowitz
    markowitz_holding = previous_markowitz_position.T@mp
    markowitz_matrix_input = markowitz_input_function(markowitz_input,date,60)
    markowitz_w = markowitz_weight(markowitz_matrix_input)
    markowitz_position = active_strategy_position(previous_markowitz_weight,markowitz_w,mp,fee_rate,cash,markowitz_holding)
    
    previous_markowitz_position = markowitz_position
    previous_markowitz_weight = markowitz_w
    if len(proposed_matrix_weight) == 0:
        proposed_matrix_weight = proposed_w
        marko_matrix_weigtht = markowitz_w
    else:
        proposed_matrix_weight = np.column_stack([proposed_matrix_weight, proposed_w])
        marko_matrix_weigtht =  np.column_stack([marko_matrix_weigtht,markowitz_w])
        
    if len(markowitz_port_value) == 0:
        markowitz_port_value.append(cash)
        proposed_port_value.append(cash)
    else:
        markowitz_port_value.append(markowitz_holding)
        proposed_port_value.append(stock_holding)
    
    initital_invest=0

In [ ]:
vni_position = []
vni_pos = 0
initital_invest = 1e7
naive_pos = np.full(shape=(len(ticker),),fill_value=0.00)
naive_position = []
for date in vni.index:
    cash = initital_invest + monthly_invested
    cash = cash*(1- fee_rate)
    vni_pos += cash / vni['vni'].loc[date]
    naive_pos += cash*naive_w/mp_data.loc[date].to_numpy()
    
    vni_position.append(vni_pos)
    if len(naive_position) == 0:
        naive_position  = naive_pos
    else:
        naive_position  = np.column_stack([naive_position,naive_pos])
    initital_invest = 0
vni_position = np.array(vni_position)
vni_port_value = vni_position*vni['vni'].to_numpy()
naive_port_value = naive_position*mp_data.iloc[1::].transpose().to_numpy()
naive_port_value = np.sum(naive_port_value,axis=0)


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(markowitz_port_value,label="Markowitz model")
plt.plot(proposed_port_value,label="Proposed model")
plt.plot(naive_port_value, label='Naive')
plt.plot(vni_port_value,label="VNIndex")
plt.plot(injected_capital_vector, label="capital injected")
plt.title("Active Management strategy Performance")
plt.legend()
plt.show()